Da meine bisherige Vorhersage nicht gut Umsetztbar ist, habe ich mich dazu entschieden sie zu ändern.
Meine jetztige Vorhersage, sieht folgendermassen aus:
Ich möchte die Location, die min und die max Temperatur und der ob es heute regnet eingeben. Und Anhand von dem wird ausgerechnet, ob es morgen eher regnen oder ob es eher nicht regnet.

In [4]:
import pandas as pd
dataset_weather = pd.read_csv("Datensatz LB-259.csv", on_bad_lines="skip", sep=";")


In [5]:

X = dataset_weather[["Location", "MinTemp","MaxTemp", "RainToday"]]
y = dataset_weather[["RainTomorrow"]]
X.head()

,Location,MinTemp,MaxTemp,RainToday
0,3,13.4,22.9,0.0
1,3,7.4,25.1,0.0
2,3,12.9,25.7,0.0
3,3,9.2,28.0,0.0
4,3,17.5,32.3,0.0


In [6]:
from sklearn.impute import SimpleImputer
import numpy as np
si = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
si.fit(y)
y_ = si.transform(y)
y = pd.DataFrame(y_, columns=y.columns)
len(y[y.isna().any(axis=1)])
si.fit(X)
X_ = si.transform(X)
X = pd.DataFrame(X_, columns=X.columns)
len(X[X.isna().any(axis=1)])

0

In [7]:
# 👾 TODO: Split the titanic data into suitable test and train sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size = 0.2, random_state = 42
)

len(X_train), len(X_test)

(116368, 29092)

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

algorithms = {
    "Nearest Neighbors" : KNeighborsClassifier(3),

    "Decision Tree" : DecisionTreeClassifier(max_depth=5),
    "Random Forest" : RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "Neural Net" : MLPClassifier(alpha=1, max_iter=1000),
    "Naive Bayes" : GaussianNB(),
    "LDA" : LinearDiscriminantAnalysis(),
}


for name, algorithm in algorithms.items():
    # 👾 TODO: Run each algorithm on X_train and y_train
    # using the .fit function; then evaluate it using
    # algorithm.score(X_test, y_test) and print the
    # name and score for each algorithm - which one 
    # performs the best?
    
    # You might need to use y_train.values.ravel()
    # rather than y_train in the fit()-function
    algorithm.fit(X_train, y_train.values.ravel())

    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))


Nearest Neighbors 0.77
Decision Tree 0.79
Random Forest 0.78
Neural Net 0.8
Naive Bayes 0.76
LDA 0.79


In [ ]:
!pip install sklearn

Ich habe Naive Bayes Algorithmus genommen, da alle algorithmen etwa die gleiche Wahrscheinlichkeit auf Erfolg haben. Da ich von Mitschülern gehört habe, das sie auch diesen genommen haben, habe ich mich dann für Naive Bayes entschieden. 
Der naive Bayes macht wird eingesetzt, wenn die Wahrscheinlichkeiten der Klassen anhand einer Reihe von Beobachtungen zu bestimmen sind.

In [10]:
import joblib
joblib.dump(algorithms["Naive Bayes"], 'dataset_weather_naive_bayes.joblib')
best_model = joblib.load('dataset_weather_naive_bayes.joblib')
best_model

GaussianNB()

In [41]:
prediction1 = pd.DataFrame([[30, 17.7, 24.7, 0.0]], columns=X_train.columns)
prediction1.head()

prediction2 = pd.DataFrame([[3, 22.7, 30.0, 1.0]], columns=X_train.columns)
prediction2.head()

prediction3 = pd.DataFrame([[14, 11.2, 20.0, 0.0]], columns=X_train.columns)
prediction3.head()

prediction4 = pd.DataFrame([[7, 7.7, 18.7, 1.0]], columns=X_train.columns)
prediction4.head()

,Location,MinTemp,MaxTemp,RainToday
0,7,7.7,18.7,1.0


In [51]:
lovebirds_at_sea = pd.concat([prediction1, prediction2, prediction3, prediction4])
lovebirds_at_sea.head()

,Location,MinTemp,MaxTemp,RainToday
0,30,17.7,24.7,0.0
0,3,22.7,30.0,1.0
0,14,11.2,20.0,0.0
0,7,7.7,18.7,1.0


In [50]:
predictions = best_model.predict(pd.concat([prediction1, prediction2, prediction3, prediction4]))
for values, prediction in zip(["Prediction 1: Morgen wird es ", "Prediction 2: Morgen wird es ", "Prediction 3: Morgen wird es ", "Prediction 4: Morgen wird es "], predictions):
    print(f"{values} {'nicht regnen' if prediction == 0 else 'regnen'}")

Prediction 1: Morgen wird es  nicht regnen
Prediction 2: Morgen wird es  regnen
Prediction 3: Morgen wird es  nicht regnen
Prediction 4: Morgen wird es  regnen


Meine Prediction funktioniert, jedoch hängt es sehr sehr stark nur von der Spalte RainToday ab. Da es sehr wahrscheinlich ist, dass wenn es heute regnet das es dann auch morgen regnet. Ich denke von der Location hängt es von allen Angaben am wenigsten ab. So finde ich, macht die Vorhersage nicht allzuviel sind.